In [6]:
import pandas as pd
in_path = 'in'
csv_file = f"{in_path}/awards_2026.csv"
# Read only the two airport columns to save memory
df = pd.read_csv(csv_file, usecols=['ORIGIN_AIRPORT_ABBREV','DESTINATION_AIRPORT_ABBREV'])
origins = df['ORIGIN_AIRPORT_ABBREV'].dropna().unique()
dests = df['DESTINATION_AIRPORT_ABBREV'].dropna().unique()
unique_codes = sorted(set(origins).union(set(dests)))
print(f"Found {len(unique_codes)} unique airport codes")
# show first 50 as a sample
print(unique_codes[:50])

Found 645 unique airport codes
['AAL', 'ABE', 'ABI', 'ABJ', 'ABQ', 'ABR', 'ABV', 'ABY', 'ABZ', 'ACC', 'ACT', 'ACV', 'ADA', 'ADD', 'ADL', 'ADQ', 'AEX', 'AGA', 'AGP', 'AGS', 'AKL', 'AKN', 'ALA', 'ALB', 'ALO', 'ALW', 'AMA', 'AMM', 'AMS', 'ANC', 'ANU', 'APN', 'AQP', 'ART', 'ASE', 'ATH', 'ATL', 'ATW', 'AUA', 'AUH', 'AUS', 'AVL', 'AVP', 'AZO', 'BAH', 'BAK', 'BAQ', 'BCN', 'BDA', 'BDL']


In [7]:
# Create single merged CSV of unique airports with US lat/lon
import os
in_path = 'in'
# Read iata-icao and prepare US lat/lon
iata_file = f"{in_path}/iata-icao.csv"
df = pd.read_csv(iata_file, usecols=['country_code','iata','latitude','longitude'])
us = df[df['country_code'] == 'US'].copy()
us = us[us['iata'].notna() & (us['iata'].str.len() == 3)]
us['latitude'] = pd.to_numeric(us['latitude'], errors='coerce')
us['longitude'] = pd.to_numeric(us['longitude'], errors='coerce')
us = us.dropna(subset=['latitude','longitude'])
us_iata = us[['iata','latitude','longitude']].drop_duplicates().sort_values('iata')
us_iata['iata'] = us_iata['iata'].str.upper()
# Normalize unique_codes from cell 1
unique_codes_norm = [c.strip().upper() for c in unique_codes if isinstance(c, str) and c.strip()!='']
# Filter unique airports to only those with US lat/lon
matched = us_iata[us_iata['iata'].isin(unique_codes_norm)].reset_index(drop=True)
# Write final single CSV
os.makedirs('out', exist_ok=True)
out_fp = 'out/unique_airports_with_latlon.csv'
matched.to_csv(out_fp, index=False)
print(f'Wrote {out_fp} with {len(matched)} rows')
print(matched.head(10))

Wrote out/unique_airports_with_latlon.csv with 323 rows
  iata  latitude  longitude
0  ABE   40.6521   -75.4408
1  ABI   32.4113   -99.6819
2  ABQ   35.0402  -106.6090
3  ABR   45.4491   -98.4218
4  ABY   31.5355   -84.1945
5  ACT   31.6113   -97.2305
6  ACV   40.9781  -124.1090
7  ADQ   57.7500  -152.4940
8  AEX   31.3274   -92.5498
9  AGS   33.3699   -81.9645
